# calculate ufs climatology & anomaly

In [1]:
import xarray as xr
import numpy as np

In [2]:
# ufs path
ufs_path = '/scratch/hbai/ufs_p6_daily/z500/'

In [6]:
year = 2012

In [7]:
months = ['01','02','03','04','05','06','07','08','09','10','11','12']

In [8]:
# create a list of ufs files to open
# open one year's files (reforeccasts initiated on 1st or 15th)
ufs_files = [f'{ufs_path}ufs_p6_{year}{month}01_z500.nc' for month in months]
ufs_files

['/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20120101_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20120201_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20120301_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20120401_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20120501_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20120601_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20120701_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20120801_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20120901_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20121001_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20121101_z500.nc',
 '/scratch/hbai/ufs_p6_daily/z500/ufs_p6_20121201_z500.nc']

In [9]:
# read one year's ufs files at once
ds = xr.open_mfdataset(ufs_files, combine='nested', concat_dim='month')
z500_ufs = ds.z500
z500_ufs

<xarray.DataArray 'z500' (month: 12, ncl0: 31, latitude: 721, longitude: 1440)>
dask.array<concatenate, shape=(12, 31, 721, 1440), dtype=float64, chunksize=(1, 31, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Dimensions without coordinates: month, ncl0
Attributes:
    average_op_ncl:  dim_avg_n function was applied

## calculate climotology

In [22]:
clim = []

# Jan
clim = z500_ufs[0,0:31,:,:]
# Feb
if year == 2012 or 2016: # leap year
    clim = xr.concat((clim, z500_ufs[1,0:29,:,:]), "ncl0")
else:
    clim = xr.concat((clim, z500_ufs[1,0:28,:,:]), "ncl0")
# Mar
clim = xr.concat((clim, z500_ufs[2,0:31,:,:]), "ncl0")
# Apr
clim = xr.concat((clim, z500_ufs[3,0:30,:,:]), "ncl0")
# May
clim = xr.concat((clim, z500_ufs[4,0:31,:,:]), "ncl0")
# Jun
clim = xr.concat((clim, z500_ufs[5,0:30,:,:]), "ncl0")
# Jul
clim = xr.concat((clim, z500_ufs[6,0:31,:,:]), "ncl0")
# Aug
clim = xr.concat((clim, z500_ufs[7,0:31,:,:]), "ncl0")
# Sep
clim = xr.concat((clim, z500_ufs[8,0:30,:,:]), "ncl0")
# Oct
clim = xr.concat((clim, z500_ufs[9,0:31,:,:]), "ncl0")
# Nov
clim = xr.concat((clim, z500_ufs[10,0:30,:,:]), "ncl0")
# Dec
clim = xr.concat((clim, z500_ufs[11,0:31,:,:]), "ncl0")

clim

<xarray.DataArray 'z500' (ncl0: 366, latitude: 721, longitude: 1440)>
dask.array<concatenate, shape=(366, 721, 1440), dtype=float64, chunksize=(31, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Dimensions without coordinates: ncl0
Attributes:
    average_op_ncl:  dim_avg_n function was applied

In [24]:
# write climatology to nctcdf file
clim.to_netcdf(ufs_path+"ufs_p6_"+str(year)+"z500_climatology.nc")
    

## calculate anomaly

In [26]:
# climatological mean
clim_mean = clim.mean(dim='ncl0')
clim_mean

<xarray.DataArray 'z500' (latitude: 721, longitude: 1440)>
dask.array<mean_agg-aggregate, shape=(721, 1440), dtype=float64, chunksize=(721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8

In [27]:
# anomaly
z500_ufs_anom = z500_ufs - clim_mean
z500_ufs_anom

<xarray.DataArray 'z500' (month: 12, ncl0: 31, latitude: 721, longitude: 1440)>
dask.array<sub, shape=(12, 31, 721, 1440), dtype=float64, chunksize=(1, 31, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Dimensions without coordinates: month, ncl0

In [ ]:
# write anomaly to nctcdf files
for i, month in enumerate(months):
    z500_ufs_anom[i].to_netcdf(ufs_path+"ufs_p6_"+str(year)+month+"01_z500_anomaly.nc")